In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats

In [20]:
aucs = pd.read_csv("AUCs.txt", sep = '\t', header = 0)
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


# #2

In [25]:
first_name = 'C4.5'
second_name = 'C4.5+m'
first_name + second_name

'C4.5C4.5+m'

In [36]:
n = len(aucs.columns)
p_values = {}

for i in range(1, n):
    for j in range(i+ 1, n):
        first_name, second_name = aucs.columns[i], aucs.columns[j]
        p_val = stats.wilcoxon(aucs[first_name], aucs[second_name])[1]
        p_values[first_name + ' + ' + second_name] = p_val

p_values

{'C4.5 + C4.5+m': 0.01075713311978963,
 'C4.5 + C4.5+cf': 0.861262330095348,
 'C4.5 + C4.5+m+cf': 0.015874359307532084,
 'C4.5+m + C4.5+cf': 0.05432871367198416,
 'C4.5+m + C4.5+m+cf': 0.3278256758446406,
 'C4.5+cf + C4.5+m+cf': 0.025313519968766574}

In [37]:
min(p_values, key = p_values.get)

'C4.5 + C4.5+m'

# #3

In [45]:
sorted_p_values = {}
sorted_keys = sorted(p_values, key = p_values.get)

for w in sorted_keys:
    sorted_p_values[w] = p_values[w]
sorted_p_values

{'C4.5 + C4.5+m': 0.01075713311978963,
 'C4.5 + C4.5+m+cf': 0.015874359307532084,
 'C4.5+cf + C4.5+m+cf': 0.025313519968766574,
 'C4.5+m + C4.5+cf': 0.05432871367198416,
 'C4.5+m + C4.5+m+cf': 0.3278256758446406,
 'C4.5 + C4.5+cf': 0.861262330095348}

In [49]:
significant_pairs = {(k, v) for k, v in sorted_p_values.items() if v <= 0.05}
len(significant_pairs)

3

# #4

In [54]:
from statsmodels.sandbox.stats.multicomp import multipletests 

In [55]:
scores = pd.DataFrame.from_dict(p_values, orient = 'index', columns = ['p'])
scores

,p
C4.5 + C4.5+m,0.010757
C4.5 + C4.5+cf,0.861262
C4.5 + C4.5+m+cf,0.015874
C4.5+m + C4.5+cf,0.054329
C4.5+m + C4.5+m+cf,0.327826
C4.5+cf + C4.5+m+cf,0.025314


In [59]:
reject_bh, p_bh, _, _ = multipletests(scores.p, alpha = 0.05, method = 'fdr_bh')
scores['p_bh'] = p_bh
scores['reject_bj'] = reject_bh
scores

,p,p_holm,reject_holm,p_bh,reject_bj
C4.5 + C4.5+m,0.010757,0.064543,False,0.047623,True
C4.5 + C4.5+cf,0.861262,0.861262,False,0.861262,False
C4.5 + C4.5+m+cf,0.015874,0.079372,False,0.047623,True
C4.5+m + C4.5+cf,0.054329,0.162986,False,0.081493,False
C4.5+m + C4.5+m+cf,0.327826,0.655651,False,0.393391,False
C4.5+cf + C4.5+m+cf,0.025314,0.101254,False,0.050627,False


In [ ]:
reject, p_benj, _, _ = multipletests(scores.p, alpha = 0.05, method = 'holm')
scores['p_holm'] = p_holm
scores['reject_holm'] = reject
scores